# Primary analyses of our sequences

In [13]:
import os
import sys
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Define the data directory
data_dir = '/home/jovyan/FunGut2/full-pipeline/data'

In [ ]:
import os

In [ ]:
!pwd

In [99]:
!qiime sample-classifier classify-samples --help

Usage: qiime sample-classifier classify-samples [OPTIONS]

  Predicts a categorical sample metadata column using a supervised learning
  classifier. Splits input data into training and test sets. The training set
  is used to train and test the estimator using a stratified k-fold cross-
  validation scheme. This includes optional steps for automated feature
  extraction and hyperparameter optimization. The test set validates
  classification accuracy of the optimized estimator. Outputs classification
  results for test set. For more details on the learning algorithm, see
  http://scikit-learn.org/stable/supervised_learning.html

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency | RelativeFrequency |
    PresenceAbsence | Composition]
                          Feature table containing all features that should
                          be used for target prediction.            [required]
Parameters:
  --m-metadata-file METADATA
  --m-metadata-column COLUMN  MetadataColumn[Categorical]


In [ ]:
!pwd

In [237]:
! rm -r machLearning

rm: cannot remove 'machLearning': No such file or directory


In [238]:
!qiime sample-classifier classify-samples \
    --i-table $data_dir/feature_tables_dada/filtered-feature-table.qza \
    --m-metadata-file $data_dir/metadata/fungut_metadata_processed.tsv \
    --m-metadata-column country_sample \
    --p-random-state 42 \
    --p-parameter-tuning \
    --p-estimator RandomForestClassifier \
    --output-dir ./machLearning/

Plugin error from sample-classifier:

  You have chosen to predict a metadata column that contains one or more values that match only one sample. For proper stratification of data into training and test sets, each class (value) must contain at least two samples. This is a requirement for classification problems, but stratification can be disabled for regression by setting stratify=False. Alternatively, remove all samples that bear a unique class label for your chosen metadata column. Note that disabling stratification can negatively impact predictive accuracy for small data sets.

Debug info has been saved to /tmp/qiime2-q2cli-err-hbw9r7rf.log


In [235]:
Visualization.load('./machLearning/accuracy_results.qzv')

ValueError: machLearning/accuracy_results.qzv does not exist.

# Self made Classifier

In [179]:
import os
import sys
import seaborn as sns
import pandas as pd
import numpy as np
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import SparsePCA

# Define the data directory
data_dir = '/home/jovyan/FunGut2/full-pipeline/data'

In [ ]:
!pip show seaborn

In [ ]:
!pip install --upgrade seaborn

### export feature table and load as data frame

In [ ]:
!qiime tools export \
        --input-path $data_dir/feature_tables_dada/filtered-feature-table.qza \
        --output-path $data_dir/feature_tables_dada/exported-feature-table

In [4]:
from biom import load_table
import pandas as pd

# Load the .biom file
biom_table = load_table(f'{data_dir}/feature_tables_dada/exported-feature-table/feature-table.biom')

# Convert the biom table to a pandas DataFrame
df = biom_table.to_dataframe()

# Display the DataFrame
df.head()

,ERR5327198,ERR5327199,ERR5327266,ERR5327282,ERR5327284,ERR5327285,ERR5327287,ERR5327288,ERR5327289,ERR5327300,...,ERR5327586,ERR5327587,ERR5327591,ERR5327592,ERR5327596,ERR5327599,ERR5327604,ERR5327605,ERR5327615,ERR5327620
25bc419d2059b2ae08c91108b86b71e6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f53f50aea8a5462421d845b18e60151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192eb2ce554d1b2e5e7c08135413e10b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5422.0,0,0,0,0,0,0
f03e4889f73ea08958135b0757e539dd,8010.0,0,0,0,0,0,0,0,0,51.0,...,0,0,0,0,0,0,0,9.0,0,0
a112f028910db87dfd35a3a916544d74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
meta = pd.read_csv(f'{data_dir}/metadata/fungut_metadata_processed.tsv', sep='\t')

### Build Classifier

In [226]:
# Example of loading your feature table (from QIIME 2 export) and metadata
X = df.T  # Your feature table (e.g., a pandas DataFrame)
y = meta.country_sample  # Your target labels (metadata column)

In [227]:
y = y[y != 'Not provided']
y.unique()
X = X.iloc[y.index]

#preprocess data
X = np.log1p(X) 

In [228]:
X = X.to_numpy()
y = y.to_numpy()

#spca = SparsePCA(n_components=100, random_state=42)
#X = spca.fit_transform(X)

# Encode the target labels
#le = LabelEncoder()
#y = le.fit_transform(y)  # Converts strings to integers

In [221]:
print(X.max())

11.407020356604264


In [231]:
#scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# Stratified K-Fold Cross-validation (ensures each fold has the same class distribution)
kf = StratifiedKFold(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y), start=1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Apply SMOTE to the training set --> create new datapoints of minority group (disease)
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    #model = RandomForestClassifier(class_weight='balanced', random_state=42)
    #model.fit(X_train, y_train)

    model = xgb.XGBClassifier(scale_pos_weight=scale_pos_weight, use_label_encoder=True, eval_metric='logloss')
    model.fit(X_train, y_train)
    
    #model = LogisticRegression(solver='lbfgs', class_weight='balanced', random_state=42)
    #model.fit(X_train_smote, y_train_smote)
    
    # Evaluate model
    y_pred = model.predict(X_test)
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Ratio: {accuracy_score(y_test, y_pred)/((len(y_test) - sum(y_test))/len(y_test))}')
    
    # Get class distribution using numpy.unique()
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
    # Print class distributions
    print(f"True class distribution: {dict(zip(unique_test, counts_test))}")
    print(f"Predicted class distribution: {dict(zip(unique_pred, counts_pred))}")
    
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Plot the confusion matrix as a heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y),
               annot_kws={"size": 14})  # Increase font size for annotations
    plt.title(f'Confusion Matrix for Fold {fold}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.show()

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 1, n_samples = 1

In [ ]:
cm = np.array(cm)

In [ ]:
print(cm)